In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# read files from data
articles = pd.read_csv('data/articles.csv.zip')
customers = pd.read_csv('data/customers.csv.zip')
transactions = pd.read_csv('data/transactions_train.csv.zip')

In [8]:
# cleanup transactions
transactions_cleaned = transactions.copy()

# convert date to datetime
transactions_cleaned['t_dat'] = pd.to_datetime(transactions_cleaned['t_dat'])

# price multiplied by 590 to unmask the real price
transactions_cleaned['price'] = transactions_cleaned['price'] * 590

transactions_cleaned = transactions_cleaned.merge(articles[['article_id','product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'index_group_name', 'section_name']], how='left', on='article_id')

# Extract month, day of week, and week of year from t_dat
transactions_cleaned['month'] = transactions_cleaned['t_dat'].dt.month
transactions_cleaned['day_of_week'] = transactions_cleaned['t_dat'].dt.dayofweek
transactions_cleaned['week_of_year'] = transactions_cleaned['t_dat'].dt.isocalendar().week

# Unique products
transactions_cleaned['unique_products'] = transactions_cleaned.groupby('customer_id')['article_id'].transform('nunique')

transactions_cleaned.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,index_group_name,section_name,month,day_of_week,week_of_year,unique_products
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,29.99,2,Underwear body,Underwear,Solid,Black,Dark,Ladieswear,Womens Lingerie,9,3,38,14
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,17.99,2,Bra,Underwear,Solid,Light Pink,Dusty Light,Ladieswear,Womens Lingerie,9,3,38,14
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,8.99,2,Sweater,Garment Upper body,Melange,Pink,Medium Dusty,Divided,Divided Selected,9,3,38,110
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,9.99,2,Sweater,Garment Upper body,Melange,Pink,Medium,Ladieswear,Womens Everyday Collection,9,3,38,110
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,9.99,2,Sweater,Garment Upper body,Melange,Dark Green,Dark,Ladieswear,Womens Everyday Collection,9,3,38,110


In [9]:
# cleanup and feature engineering on customers customers_cleaned

# Bin the age and create new column
customers_cleaned = customers.copy()
bins = [18, 24, 34, 44, 54, 64, 100]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
customers_cleaned['age_bin'] = pd.cut(customers_cleaned['age'], bins=bins, labels=labels, right=False)

def categorize_age(age):
    if age < 25:
        return 'Young'
    elif age < 50:
        return 'Adult'
    else:
        return 'Senior'

# Apply the function to the 'age' column
customers_cleaned['age_group'] = customers_cleaned['age'].apply(categorize_age)

# Transaction count per customer
customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby(['customer_id'])['article_id'].count(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"article_id":"Overall_transaction_count"}, axis='columns', inplace=True)

# Num of transactions through sales channel ID per customer
customers_cleaned = customers_cleaned.merge(transactions_cleaned[transactions_cleaned['sales_channel_id'] == 1].groupby(['customer_id'])['article_id'].count(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"article_id":"Overall_transaction_count_channel_1"}, axis='columns', inplace=True)
customers_cleaned = customers_cleaned.merge(transactions_cleaned[transactions_cleaned['sales_channel_id'] == 2].groupby(['customer_id'])['article_id'].count(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"article_id":"Overall_transaction_count_channel_2"}, axis='columns', inplace=True)

# Last transaction date
customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby('customer_id')['t_dat'].max(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"t_dat":"Last_transaction_date"}, axis='columns', inplace=True)

# Total amount bought through the period
customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby('customer_id')['price'].sum(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"price":"Overall_transacted"}, axis='columns', inplace=True)

In [11]:
def get_most_bought_item(df, groupby_cols, value_col, name_suffix):
    most_bought_item = (df.groupby(groupby_cols)[value_col]
                          .sum()
                          .groupby('customer_id')
                          .idxmax()
                          .apply(lambda x: x[1]))
    return most_bought_item.rename(name_suffix)

In [12]:
# Compute most bought items for each category
categories = ['product_type_name', 'graphical_appearance_name', 
              'perceived_colour_value_name', 'section_name']
suffixes = ['Product_type_bought_most', 'Graphic_bought_most',
            'Perceived_color_bought_most', 'Section_bought_most']
most_bought_items = [get_most_bought_item(transactions_cleaned, 
                                          ['customer_id', category], 
                                          'price', suffix)
                     for category, suffix in zip(categories, suffixes)]

# Merge results with processed_customers
for most_bought_item in most_bought_items:
    customers_cleaned = customers_cleaned.merge(most_bought_item, 
                                                    how='left', 
                                                    on='customer_id')

In [14]:
customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby('customer_id')['t_dat'].min(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"t_dat":"Earliest_transaction_date"}, axis='columns', inplace=True)

latest_date = pd.to_datetime(transactions_cleaned['t_dat'].max())

customers_cleaned['Recency'] = latest_date - customers_cleaned['Last_transaction_date']
customers_cleaned['Tenure'] = customers_cleaned['Last_transaction_date'] - customers_cleaned['Earliest_transaction_date']

customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby('customer_id')['article_id'].nunique(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"article_id":"Transaction_count_revised"}, axis='columns', inplace=True)

customers_cleaned['Recency_days'] = customers_cleaned['Recency'].dt.days
customers_cleaned['Tenure_days'] = customers_cleaned['Tenure'].dt.days

customers_cleaned = customers_cleaned.merge(transactions_cleaned.groupby('customer_id')['article_id'].nunique(), how='left', on='customer_id')
customers_cleaned.rename(mapper={"article_id":"Unique_items"}, axis='columns', inplace=True)
customers_cleaned['Unique_items'].fillna(0, inplace=True)

In [16]:
customers_cleaned.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_bin,age_group,Overall_transaction_count,...,Graphic_bought_most,Perceived_color_bought_most,Section_bought_most,Earliest_transaction_date,Recency,Tenure,Transaction_count_revised,Recency_days,Tenure_days,Unique_items
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,45-54,Adult,21.0,...,Solid,Dark,Womens Tailoring,2018-12-27,17 days,618 days,19.0,17.0,618.0,19.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,25-34,Adult,86.0,...,Solid,Dark,"Womens Swimwear, beachwear",2018-09-21,76 days,656 days,64.0,76.0,656.0,64.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,25-34,Young,18.0,...,Solid,Dark,Men Suits & Tailoring,2018-09-20,7 days,726 days,14.0,7.0,726.0,14.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,55-64,Senior,2.0,...,Solid,Dark,Ladies H&M Sport,2019-06-09,471 days,0 days,2.0,471.0,0.0,2.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,45-54,Senior,13.0,...,Solid,Dark,Womens Lingerie,2018-10-12,41 days,670 days,12.0,41.0,670.0,12.0


In [17]:
transactions_cleaned.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,index_group_name,section_name,month,day_of_week,week_of_year,unique_products
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,29.99,2,Underwear body,Underwear,Solid,Black,Dark,Ladieswear,Womens Lingerie,9,3,38,14
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,17.99,2,Bra,Underwear,Solid,Light Pink,Dusty Light,Ladieswear,Womens Lingerie,9,3,38,14
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,8.99,2,Sweater,Garment Upper body,Melange,Pink,Medium Dusty,Divided,Divided Selected,9,3,38,110
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,9.99,2,Sweater,Garment Upper body,Melange,Pink,Medium,Ladieswear,Womens Everyday Collection,9,3,38,110
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,9.99,2,Sweater,Garment Upper body,Melange,Dark Green,Dark,Ladieswear,Womens Everyday Collection,9,3,38,110


In [18]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [19]:
# save to pickle file to preserve data
customers_cleaned.to_pickle('customers_cleaned.pkl')
transactions_cleaned.to_pickle('transactions_cleaned.pkl')